# 🧠 Hybrid Brain Tumor Detection System
## BME 271D Final Project - Ege, Max, Sasha

**Combining FFT and Spatial Methods for Robust Detection**

In [ ]:
!pip install -q numpy matplotlib scipy scikit-image pandas
!wget -q https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/tumor_segmentation.py
!wget -q https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/generate_realistic_tumors.py
!mkdir -p data/images data/masks
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_001.png
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_002.png
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_003.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_001.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_002.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_003.png

import tumor_segmentation as ts
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as PILImage
from google.colab import files
from scipy import ndimage
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects, binary_closing, binary_opening, disk

def load_image_safe(f):
    return np.array(PILImage.open(f).convert('L')).astype(np.float64) / 255.0

def load_mask_safe(f):
    return (np.array(PILImage.open(f).convert('L')) > 127).astype(np.uint8)

def remove_peripheral(mask, image, border_frac=0.12):
    """Remove skull and peripheral structures."""
    h, w = mask.shape
    border_h, border_w = int(h * border_frac), int(w * border_frac)
    
    internal = np.zeros_like(mask, dtype=bool)
    internal[border_h:-border_h, border_w:-border_w] = True
    
    labeled = label(mask)
    cleaned = np.zeros_like(mask, dtype=bool)
    
    for region in regionprops(labeled):
        region_pixels = (labeled == region.label)
        internal_pixels = np.logical_and(region_pixels, internal).sum()
        
        # Must be >60% internal
        if internal_pixels / region.area > 0.6:
            cleaned[region_pixels] = True
    
    return cleaned

def fft_detection(image, sensitivity=0.5):
    """FFT-based detection."""
    mean_val, std_val = image.mean(), image.std()
    results = {}
    
    # High-pass
    try:
        hp_img, _, _ = ts.filter_pipeline(image, 'hp', cutoff_radius=25)
        hp_img = (hp_img - hp_img.min()) / (hp_img.max() - hp_img.min() + 1e-8)
        hp_mask = hp_img > np.percentile(hp_img, 87 - sensitivity * 7)
        hp_mask = np.logical_and(hp_mask, image > mean_val + 0.5 * std_val)
        hp_mask = binary_closing(hp_mask, disk(2))
        hp_mask = remove_small_objects(hp_mask, min_size=80)
        hp_mask = remove_peripheral(hp_mask, image)
        results['FFT_HighPass'] = hp_mask
    except:
        results['FFT_HighPass'] = np.zeros_like(image, dtype=bool)
    
    # Band-pass
    try:
        bp_img, _, _ = ts.filter_pipeline(image, 'bp', r1=10, r2=50)
        bp_img = (bp_img - bp_img.min()) / (bp_img.max() - bp_img.min() + 1e-8)
        bp_mask = bp_img > np.percentile(bp_img, 83 - sensitivity * 7)
        bp_mask = np.logical_and(bp_mask, image > mean_val + 0.5 * std_val)
        bp_mask = binary_closing(bp_mask, disk(2))
        bp_mask = remove_small_objects(bp_mask, min_size=80)
        bp_mask = remove_peripheral(bp_mask, image)
        results['FFT_BandPass'] = bp_mask
    except:
        results['FFT_BandPass'] = np.zeros_like(image, dtype=bool)
    
    # Combined: OR (either method)
    results['FFT_Combined'] = np.logical_or(results['FFT_HighPass'], results['FFT_BandPass'])
    return results

def blob_detection(image, sensitivity=0.5):
    """Blob detection for bright masses."""
    h, w = image.shape
    mean_val, std_val = image.mean(), image.std()
    
    # Very bright regions
    bright_thresh = mean_val + (2.2 - sensitivity * 0.4) * std_val
    percentile_thresh = np.percentile(image, 92 - sensitivity * 5)
    
    blob_mask = np.logical_and(image > bright_thresh, image > percentile_thresh)
    blob_mask = binary_opening(blob_mask, disk(2))
    blob_mask = binary_closing(blob_mask, disk(3))
    blob_mask = remove_small_objects(blob_mask, min_size=120)
    blob_mask = remove_peripheral(blob_mask, image, border_frac=0.12)
    
    # Filter by shape
    labeled = label(blob_mask)
    filtered = np.zeros_like(blob_mask)
    
    for region in regionprops(labeled, intensity_image=image):
        if region.area < 100 or region.area > h * w * 0.25:
            continue
        circularity = 4 * np.pi * region.area / (region.perimeter ** 2 + 1e-8)
        if circularity < 0.12:
            continue
        if region.solidity < 0.65:
            continue
        if region.mean_intensity < mean_val + std_val:
            continue
        filtered[labeled == region.label] = True
    
    return {'Blob_Detection': filtered}

def hybrid_detection(image, sensitivity=0.5):
    """Hybrid ensemble with BALANCED voting."""
    fft_res = fft_detection(image, sensitivity)
    blob_res = blob_detection(image, sensitivity)
    all_res = {**fft_res, **blob_res}
    
    # Voting: FFT gets 1.5 votes, Blob gets 1 vote
    # Need 1.5+ votes (can be FFT alone OR FFT+Blob)
    vote_map = (fft_res['FFT_Combined'].astype(float) * 1.5 +
                blob_res['Blob_Detection'].astype(float) * 1.0)
    
    combined = vote_map >= (1.4 - sensitivity * 0.2)
    combined = binary_closing(combined, disk(2))
    combined = remove_small_objects(combined, min_size=100)
    
    # Final shape filter
    labeled = label(combined)
    final = np.zeros_like(combined)
    
    for region in regionprops(labeled):
        if region.eccentricity > 0.96 or region.solidity < 0.65:
            continue
        final[labeled == region.label] = True
    
    all_res['Hybrid_Combined'] = final
    return all_res

def analyze_detection(image, mask):
    """Analyze detection."""
    h, w = image.shape
    total = h * w
    tumor_px = mask.sum()
    area_pct = (tumor_px / total) * 100
    
    if tumor_px < total * 0.004:  # <0.4%
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    coords = np.where(mask)
    if len(coords[0]) == 0:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    center = (int(np.mean(coords[0])), int(np.mean(coords[1])))
    labeled = label(mask)
    regions = regionprops(labeled, intensity_image=image)
    
    if len(regions) == 0:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    largest = max(regions, key=lambda x: x.area)
    
    # Confidence
    if 0.4 <= area_pct <= 12:
        size_score = 1.0
    elif area_pct < 0.4:
        size_score = area_pct / 0.4
    else:
        size_score = max(0, 1 - (area_pct - 12) / 18)
    
    circularity = 4 * np.pi * largest.area / (largest.perimeter ** 2 + 1e-8)
    shape_score = min(circularity * 2.5, 1.0)
    
    tumor_int = image[mask].mean()
    bg_int = image[~mask].mean() if (~mask).any() else 0
    contrast_score = min(max((tumor_int - bg_int) * 4, 0), 1.0)
    
    confidence = size_score * 0.3 + shape_score * 0.3 + contrast_score * 0.4
    
    if confidence < 0.28:  # Lower threshold
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    return {
        'detected': True,
        'area_percent': area_pct,
        'center': center,
        'confidence': np.clip(confidence, 0, 1),
        'num_regions': len(regions)
    }

image, mask = None, None
print('✅ Ready! Balanced hybrid detection')

In [ ]:
# Upload
uploaded = files.upload()
if uploaded:
    image = load_image_safe(list(uploaded.keys())[0])
    mask = None
    plt.figure(figsize=(8,8))
    plt.imshow(image, cmap='gray')
    plt.title('Brain MRI')
    plt.axis('off')
    plt.show()

In [ ]:
# FFT
if image is not None:
    F_shift, mag = ts.compute_fft_spectrum(image)
    ts.visualize_frequency_spectrum(image, F_shift)
    plt.show()

In [ ]:
# Detection
if image is not None:
    results = hybrid_detection(image, sensitivity=0.5)
    methods = ['FFT_HighPass', 'FFT_BandPass', 'FFT_Combined', 'Blob_Detection', 'Hybrid_Combined']
    
    fig, axes = plt.subplots(2, 3, figsize=(16, 11))
    axes = axes.ravel()
    
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original', fontsize=13, fontweight='bold')
    axes[0].axis('off')
    
    for idx, method in enumerate(methods, 1):
        mask_m = results[method]
        analysis = analyze_detection(image, mask_m)
        
        axes[idx].imshow(image, cmap='gray')
        axes[idx].imshow(mask_m, cmap='Reds', alpha=0.5)
        
        if analysis['center']:
            r, c = analysis['center']
            axes[idx].plot(c, r, 'g+', markersize=20, markeredgewidth=3)
        
        status = f"DETECTED\n{analysis['area_percent']:.1f}%" if analysis['detected'] else "Not detected"
        axes[idx].set_title(f"{method}\n{status}", fontsize=11, fontweight='bold')
        axes[idx].axis('off')
    
    plt.suptitle('Hybrid Detection', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print('\n' + '='*70)
    print(f'{"Method":<20} {"Detected":<12} {"Area %":<12}')
    print('='*70)
    for method in methods:
        analysis = analyze_detection(image, results[method])
        det = '✓ YES' if analysis['detected'] else '✗ No'
        area = f"{analysis['area_percent']:.2f}%" if analysis['detected'] else '-'
        print(f'{method:<20} {det:<12} {area:<12}')
    print('='*70)

In [ ]:
# Final Verdict
if image is not None and 'results' in dir():
    final_mask = results['Hybrid_Combined']
    final_analysis = analyze_detection(image, final_mask)
    
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(image, cmap='gray')
    if final_analysis['detected']:
        axes[1].imshow(final_mask, cmap='Reds', alpha=0.5)
        if final_analysis['center']:
            r, c = final_analysis['center']
            axes[1].plot(c, r, 'g+', markersize=30, markeredgewidth=4)
    axes[1].set_title('Detection', fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    axes[2].axis('off')
    
    if final_analysis['detected']:
        if final_analysis['confidence'] > 0.65:
            status, verdict = '⚠️ TUMOR DETECTED', '🔴 HIGH CONFIDENCE'
        elif final_analysis['confidence'] > 0.4:
            status, verdict = '❓ POSSIBLE TUMOR', '🟡 MEDIUM CONFIDENCE'
        else:
            status, verdict = '❓ SUSPICIOUS', '🟠 LOW CONFIDENCE'
    else:
        status, verdict = '✅ NO TUMOR', '🟢 Normal brain'
    
    text = f"""
    
    {status}
    
    ─────────────────────
    
    Confidence: {final_analysis['confidence']*100:.1f}%
    Area: {final_analysis['area_percent']:.2f}%
    
    Method: Hybrid Ensemble
    
    ─────────────────────
    
    {verdict}
    """
    
    axes[2].text(0.05, 0.5, text, fontsize=13, verticalalignment='center',
                fontfamily='monospace', 
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    axes[2].set_title('Diagnosis', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print('\n' + '='*60)
    print(f'  {verdict}')
    print('='*60)